In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

In [0]:
!pip install gluonnlp
!pip install mxnet
!pip install -U sentence-transformers

In [0]:
import gluonnlp as nlp
import numpy as np

In [0]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [0]:
news_dataset = nlp.data.TSVDataset('gdrive/My Drive/Colab/News Subjectivity/train.txt', num_discard_samples=1)

In [0]:
check = True
for i in range(len(news_dataset)):
  if 'I' in news_dataset[i] or news_dataset[i][1] == '':
    continue
  else:
    character_vector = model.encode([news_dataset[i][0]])[0]
    character_vector = np.expand_dims(character_vector, axis=0)
    if check:
      char_vectors = character_vector.copy()
      check = False
    else:
      char_vectors = np.concatenate((char_vectors, character_vector), axis=0)

In [0]:
news_dataset_test = nlp.data.TSVDataset('gdrive/My Drive/Colab/News Subjectivity/test.txt', num_discard_samples=1)

In [0]:
check = True
for i in range(len(news_dataset_test)):
  if 'I' in news_dataset_test[i] or news_dataset_test[i][1] == '':
    continue
  else:
    character_vector = model.encode([news_dataset_test[i][0]])[0]
    character_vector = np.expand_dims(character_vector, axis=0)
    if check:
      char_vectors_test = character_vector.copy()
      check = False
    else:
      char_vectors_test = np.concatenate((char_vectors_test, character_vector), axis=0)

Determine labelings

In [0]:
labels = []
check = True
for i in range(len(news_dataset)):
  if 'I' in news_dataset[i] or news_dataset[i][1] == '':
    continue
  else:
    if news_dataset[i][1] == '':
      print(news_dataset[i])
    labels.append(news_dataset[i][1].strip())

In [0]:
labels_test = []
check = True
for i in range(len(news_dataset_test)):
  if 'I' in news_dataset_test[i] or news_dataset_test[i][1] == '':
    continue
  else:
    if news_dataset_test[i][1] == '':
      print(news_dataset_test[i])
    labels_test.append(news_dataset_test[i][1].strip())

In [0]:
for i, e in enumerate(labels):
  if int(e) > 2:
    labels[i] = str(2)

In [0]:
for i, e in enumerate(labels_test):
  if int(e) > 2:
    labels_test[i] = str(2)

In [109]:
{x:labels.count(x) for x in labels}

{'1': 5472, '2': 5907}

In [110]:
{x:labels_test.count(x) for x in labels_test}

{'1': 671, '2': 586}

In [114]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=10, alpha=1e-4,
                    solver='sgd', verbose=10, random_state=1,
                    learning_rate_init=.1)
mlp.fit(char_vectors, np.array(labels))

Iteration 1, loss = 0.58156650
Iteration 2, loss = 0.53646978
Iteration 3, loss = 0.51921770
Iteration 4, loss = 0.50681224
Iteration 5, loss = 0.48759272
Iteration 6, loss = 0.47105902
Iteration 7, loss = 0.45755695
Iteration 8, loss = 0.44201493
Iteration 9, loss = 0.42341342
Iteration 10, loss = 0.40852628


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50,), learning_rate='constant',
              learning_rate_init=0.1, max_fun=15000, max_iter=10, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='sgd', tol=0.0001,
              validation_fraction=0.1, verbose=10, warm_start=False)

In [115]:
mlp.score(char_vectors_test, np.array(labels_test))

0.7247414478918058

In [116]:
from joblib import dump, load
dump(mlp, 'gdrive/My Drive/Colab/News Subjectivity/neural_classifier.joblib')

['gdrive/My Drive/Colab/News Subjectivity/neural_classifier.joblib']